In [15]:
# Load libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
#from sklearn.metrics import accuracy_score, classification_reportt

## Importing the Data

In [17]:
movie_data = pd.read_csv(r"/Users/krummelha/Desktop/J2024/Project2/wiki_movie_plots_deduped.csv")
movie_data.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34886 entries, 0 to 34885
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Release Year      34886 non-null  int64 
 1   Title             34886 non-null  object
 2   Origin/Ethnicity  34886 non-null  object
 3   Director          34886 non-null  object
 4   Cast              33464 non-null  object
 5   Genre             34886 non-null  object
 6   Wiki Page         34886 non-null  object
 7   Plot              34886 non-null  object
dtypes: int64(1), object(7)
memory usage: 2.1+ MB


## Cleaning the Data (Dropping)

In [18]:
# Select only films that are American in origin
us_movies = movie_data[movie_data['Origin/Ethnicity'] == 'American']

# Drop unwanted columns (Wiki.Page, Director, Cast)
us_movies = us_movies.drop(columns=['Wiki Page', 'Director', 'Cast'])

# Print the summary of the resulting DataFrame
us_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17377 entries, 0 to 17376
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Release Year      17377 non-null  int64 
 1   Title             17377 non-null  object
 2   Origin/Ethnicity  17377 non-null  object
 3   Genre             17377 non-null  object
 4   Plot              17377 non-null  object
dtypes: int64(1), object(4)
memory usage: 814.5+ KB


In [19]:
# Fetch Title and Plot columns
finaldata = us_movies[["Title", "Plot"]]
# Setting the movie title as index
finaldata = finaldata.set_index('Title')
finaldata.head()

,Plot
Title,
Kansas Saloon Smashers,"A bartender is working at a saloon, serving dr..."
Love by the Light of the Moon,"The moon, painted with a smiling face hangs ov..."
The Martyred Presidents,"The film, just over a minute long, is composed..."
"Terrible Teddy, the Grizzly King",Lasting just 61 seconds and consisting of two ...
Jack and the Beanstalk,The earliest known adaptation of the classic f...


In [11]:
#Project two, Movie recommender

## Guiding Question
- How do we create a machine learning model that offers recommendations based on movie plot? 

## Data Background
- Data Dimensions 
- ~35,000 movies (rows) x 8 columns 

### Columns in the data
- Year
- Title
- Origin/Ethnicity (dropping everything that’s not American) 
- Director (dropping)
- Cast (dropping)
- Genre
- Wiki page (dropping) 
- Plot 

### What our cleaned data set looks like (before LDA pre-processing)
- 5 x ~17,500
- Year, Title, Origin = American, Genre, Plot

## LDA Background 
- Because our recommender will be based of text recognition and synopsis, we must have a way to identify trends within each text piece.
- LDA (Latent Dirichlet Allocation) is a topic modeling technique used for extracting topics from text documents. It finds topics that the “text” belongs to on the basis of the specific words it contains. It then groups similar topics allowing to map probability distribution over latent topics

### Term Frequency-Inverse Document Frequency
- Count each words frequency thus converting the text format to a numerical one





## Importing LDA and Tokenizer Specific Packages

## Using nltk and spaCy


In [20]:
#pip install stop-words

  Preparing metadata (setup.py) ... done
  DEPRECATION: stop-words is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Running setup.py install for stop-words ... done

[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [25]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

In [26]:
tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

## 1) dividing text into sentences


In [37]:
import spacy

# initialize spacy
nlp = spacy.load("en_core_web_sm")

In [50]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [56]:

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/krummelha/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [45]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/krummelha/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [57]:
#Tokenize

# def sent_to_words(sentences):
#     for sentence in sentences:
#             sentence = sentence.lower()(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

# data_words = list(sent_to_words(data))
# #print(data_words[:1])

#nltk.download('punkt')
#nltk.download('stopwords')

#we will not need to process data to take out unimportant words

# Download stopwords dataset
#nltk.download('stopwords')
#nltk.download('punkt')

def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    filtered_tokens = [word.lower() for word in tokens if word.isalnum() and word.lower() not in stop_words]
    return ' '.join(filtered_tokens)
        

In [58]:
finaldata['Processed_Plot'] = finaldata['Plot'].apply(preprocess_text)

In [60]:
finaldata.head()

,Plot,Processed_Plot
Title,,
Kansas Saloon Smashers,"A bartender is working at a saloon, serving dr...",bartender working saloon serving drinks custom...
Love by the Light of the Moon,"The moon, painted with a smiling face hangs ov...",moon painted smiling face hangs park night you...
The Martyred Presidents,"The film, just over a minute long, is composed...",film minute long composed two shots first girl...
"Terrible Teddy, the Grizzly King",Lasting just 61 seconds and consisting of two ...,lasting 61 seconds consisting two shots first ...
Jack and the Beanstalk,The earliest known adaptation of the classic f...,earliest known adaptation classic fairytale fi...


In [62]:
# Extract the 'Plot' column values from the DataFrame and convert to a list
finaldata = finaldata.Plot.values.tolist()

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data_words = list(sent_to_words(finaldata)) # Convert the list of sentences to a list of tokenized wordsta))

print(data_words[:1])

[['bartender', 'is', 'working', 'at', 'saloon', 'serving', 'drinks', 'to', 'customers', 'after', 'he', 'fills', 'stereotypically', 'irish', 'man', 'bucket', 'with', 'beer', 'carrie', 'nation', 'and', 'her', 'followers', 'burst', 'inside', 'they', 'assault', 'the', 'irish', 'man', 'pulling', 'his', 'hat', 'over', 'his', 'eyes', 'and', 'then', 'dumping', 'the', 'beer', 'over', 'his', 'head', 'the', 'group', 'then', 'begin', 'wrecking', 'the', 'bar', 'smashing', 'the', 'fixtures', 'mirrors', 'and', 'breaking', 'the', 'cash', 'register', 'the', 'bartender', 'then', 'sprays', 'seltzer', 'water', 'in', 'nation', 'face', 'before', 'group', 'of', 'policemen', 'appear', 'and', 'order', 'everybody', 'to', 'leave']]


In [64]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']): #'NOUN', 'ADJ', 'VERB', 'ADV'
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out # Initialize an empty list to store lemmatized te #returns the list of lemmatized textsxts

# Extract lemmatized forms, excluding pronouns, and considering specified POS tags token.lemma_ not in ['-PRON-'] else ''
#  for token in doc if token.pos_ in allowed_postags]))



In [67]:
# Initialize spacy ‘en’ model, keeping only tagger component (for efficiency)
# Run in terminal: python -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
                                
# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'VERB']) #select noun and verb
print(data_lemmatized[:2])

['bartender work serve drink customer fill man bucket beer nation follower burst assault man pull hat eye dump beer head group begin wreck bar smash fixture mirror break cash register bartender spray seltzer water nation face group policeman appear order leave', 'moon paint smile face hang park night couple walk fence learn rail look moon smile embrace moon smile get sit bench tree moon view block cause frown scene man fan woman hat moon leave sky perch shoulder see']


In [70]:
from sklearn.feature_extraction.text import CountVectorizer

In [71]:
## Document word matrix 

vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,
# minimum reqd occurences of a word 
                             stop_words='english',             
# remove stop words
                             lowercase=True,                   
# convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}')
# num chars > 3
                             # max_features=50000,             
# max number of uniq words    )
data_vectorized = vectorizer.fit_transform(data_lemmatized)

In [78]:
print(data_vectorized)

  (0, 703)	2
  (0, 10060)	1
  (0, 8199)	1
  (0, 2948)	1
  (0, 2319)	1
  (0, 5599)	2
  (0, 1176)	1
  (0, 775)	2
  (0, 6147)	2
  (0, 3698)	1
  (0, 1236)	1
  (0, 474)	1
  (0, 7181)	1
  (0, 4296)	1
  (0, 3428)	1
  (0, 2990)	1
  (0, 4324)	1
  (0, 4139)	2
  (0, 783)	1
  (0, 10084)	1
  (0, 667)	1
  (0, 8478)	1
  (0, 3621)	1
  (0, 5901)	1
  (0, 1090)	1
  :	:
  (17376, 4053)	1
  (17376, 1718)	1
  (17376, 2482)	1
  (17376, 240)	17
  (17376, 3603)	1
  (17376, 242)	1
  (17376, 1894)	1
  (17376, 5220)	1
  (17376, 7733)	15
  (17376, 2820)	1
  (17376, 1599)	1
  (17376, 8806)	1
  (17376, 5169)	1
  (17376, 2584)	1
  (17376, 6492)	1
  (17376, 7356)	1
  (17376, 354)	1
  (17376, 6095)	2
  (17376, 3936)	1
  (17376, 6093)	1
  (17376, 9025)	1
  (17376, 6714)	1
  (17376, 9802)	1
  (17376, 5377)	1
  (17376, 3828)	1


In [73]:
from sklearn.decomposition import LatentDirichletAllocation

In [74]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=20,               # Number of topics (consider changing to 13 bc of movie genre types (https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwi7rYihmtGDAxU3k2oFHZfZAwUQFnoECBoQAQ&url=https%3A%2F%2Fwww.forbes.com%2Fsites%2Fdarreonnadavis%2Farticle%2Fa-complete-list-of-movie-genres%2F&usg=AOvVaw2h63GalA0RbsXCN82zwfL9&opi=89978449))
                                      max_iter=10,               
# Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          
# Random state
                                      batch_size=128,            
# n docs in each learning iter
                                      evaluate_every = -1,       
# compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               
# Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)
print(lda_model)  # Model attributes

LatentDirichletAllocation(learning_method='online', n_components=20, n_jobs=-1,
                          random_state=100)


In [80]:
LatentDirichletAllocation(
    batch_size=128,
    doc_topic_prior=None,
    evaluate_every=-1,
    learning_decay=0.7,
    learning_method='online',
    learning_offset=10.0,
    max_doc_update_iter=100,
    max_iter=10,
    mean_change_tol=0.001,
    n_components=20,  # Change from n_topics to n_components
    n_jobs=-1,
    perp_tol=0.1,
    random_state=100,
    topic_word_prior=None,
    total_samples=1000000.0,
    verbose=0
)

LatentDirichletAllocation(learning_method='online', n_components=20, n_jobs=-1,
                          random_state=100)

In [82]:
## diagnose model perplexity

# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))
# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))
# See model parameters
print(lda_model.get_params())

Log Likelihood:  -18825847.92138279
Perplexity:  2244.260312202207
{'batch_size': 128, 'doc_topic_prior': None, 'evaluate_every': -1, 'learning_decay': 0.7, 'learning_method': 'online', 'learning_offset': 10.0, 'max_doc_update_iter': 100, 'max_iter': 10, 'mean_change_tol': 0.001, 'n_components': 20, 'n_jobs': -1, 'perp_tol': 0.1, 'random_state': 100, 'topic_word_prior': None, 'total_samples': 1000000.0, 'verbose': 0}


In [84]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV

In [85]:
# Define Search Param
search_params = {'n_components': [10, 15, 20, 25, 30], 'learning_decay': [0.5, 0.7, 0.9]}

# Init the Model
lda = LatentDirichletAllocation(max_iter=5, learning_method='online', learning_offset=50., random_state=0)

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

In [ ]:
# Best Model
best_lda_model = model.best_estimator_
# Model Parameters
print("Best Model's Params: ", model.best_params_)
# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)
# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

In [ ]:
# Create Document — Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)
# column names
topicnames = ['Topic' + str(i) for i in range(best_lda_model.n_components)]
# index names
docnames = ['Doc' + str(i) for i in range(len(data))]
# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic
# Styling
def color_green(val):
 color = 'green' if val > .1 else 'black'
 return 'color: {col}'.format(col=color)
def make_bold(val):
 weight = 700 if val > .1 else 400
 return 'font-weight: {weight}'.format(weight=weight)
# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

In [ ]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)
# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames
# View
df_topic_keywords.head()

In [ ]:
# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords
topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)
# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

In [ ]:
Topics = ["Update Version/Fix Crash Problem","Download/Internet Access","Learn and Share","Card Payment","Notification/Support", 
          "Account Problem", "Device/Design/Password", "Language/Recommend/Screen Size", "Graphic/ Game Design/ Level and Coin", "Photo/Search"]
df_topic_keywords["Topics"]=Topics
df_topic_keywords

In [ ]:
# Define function to predict topic for a given text document.
nlp = spacy.load('en', disable=['parser', 'ner'])
def predict_topic(text, nlp=nlp):
    global sent_to_words
    global lemmatization
# Step 1: Clean with simple_preprocess
    mytext_2 = list(sent_to_words(text))
# Step 2: Lemmatize
    mytext_3 = lemmatization(mytext_2, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
# Step 3: Vectorize transform
    mytext_4 = vectorizer.transform(mytext_3)
# Step 4: LDA Transform
    topic_probability_scores = best_lda_model.transform(mytext_4)
    topic = df_topic_keywords.iloc[np.argmax(topic_probability_scores), 1:14].values.tolist()
    
    # Step 5: Infer Topic
    infer_topic = df_topic_keywords.iloc[np.argmax(topic_probability_scores), -1]
    
    #topic_guess = df_topic_keywords.iloc[np.argmax(topic_probability_scores), Topics]
    return infer_topic, topic, topic_probability_scores
# Predict the topic
mytext = ["Very Useful in diabetes age 30. I need control sugar. thanks Good deal"]
infer_topic, topic, prob_scores = predict_topic(text = mytext)
print(topic)
print(infer_topic)

In [ ]:
def apply_predict_topic(text):
 text = [text]
 infer_topic, topic, prob_scores = predict_topic(text = text)
 return(infer_topic)
df["Topic_key_word"]= df['Translated_Review'].apply(apply_predict_topic)
df.head()

In [ ]:
df.groupby('Topic_key_word').nunique()

In [ ]:
# output prediction result

df.to_csv('googlePlayStore_review_LDA.csv')

In [ ]:
# clustering 

# Construct the k-means clusters
from sklearn.cluster import KMeans
clusters = KMeans(n_clusters=15, random_state=100).fit_predict(lda_output)
# Build the Singular Value Decomposition(SVD) model
svd_model = TruncatedSVD(n_components=2)  # 2 components
lda_output_svd = svd_model.fit_transform(lda_output)
# X and Y axes of the plot using SVD decomposition
x = lda_output_svd[:, 0]
y = lda_output_svd[:, 1]
# Weights for the 15 columns of lda_output, for each component
print("Component's weights: \n", np.round(svd_model.components_, 2))
# Percentage of total information in 'lda_output' explained by the two components
print("Perc of Variance Explained: \n", np.round(svd_model.explained_variance_ratio_, 2))

In [ ]:
# Plot
plt.figure(figsize=(12, 12))
plt.scatter(x, y, c=clusters)
plt.xlabel('Component 2')
plt.xlabel('Component 1')
plt.title("Segregation of Topic Clusters", )

In [ ]:
# most similar are ones with smallest distance

from sklearn.metrics.pairwise import euclidean_distances
nlp = spacy.load('en', disable=['parser', 'ner'])
def similar_documents(text, doc_topic_probs, documents = data, nlp=nlp, top_n=5, verbose=False):
    topic, x  = predict_topic(text)
    dists = euclidean_distances(x.reshape(1, -1), doc_topic_probs)[0]
    doc_ids = np.argsort(dists)[:top_n]
    if verbose:        
        print("Topic KeyWords: ", topic)
        print("Topic Prob Scores of text: ", np.round(x, 1))
        print("Most Similar Doc's Probs:  ", np.round(doc_topic_probs[doc_ids], 1))
    return doc_ids, np.take(documents, doc_ids)

In [ ]:
# get similar documents: 

mytext = [“(movie name)”]
doc_ids, docs = similar_documents(text=mytext, doc_topic_probs=lda_output, documents = data, top_n=1, verbose=True)
print('\n', docs[0][:500])
print()